<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Ergontropic_stocktrading2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# Define function to calculate entropy
def entropy(x):
    return -np.sum(x * np.log(x))

# Load historical data
#ticker = "AAPL"
#ticker = "GOOGL"
#ticker = "MSFT"
ticker = "INTC"

start_date = dt.datetime.now() - dt.timedelta(days=365)
end_date = dt.datetime.now()
data = yf.download(ticker, start=start_date, end=end_date)

# Define features
data["returns"] = np.log(data["Adj Close"]) - np.log(data["Adj Close"].shift(1))
data["volatility"] = data["returns"].rolling(window=20).std()
data["momentum"] = data["returns"].rolling(window=10).sum()
data.dropna(inplace=True)

# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(data[["volatility", "momentum"]])

# Train logistic regression model
target = np.where(data["returns"].shift(-1) > 0, 1, 0)
model = LogisticRegression(random_state=0).fit(features, target)

# Calculate probability of positive return
prob = model.predict_proba(features)[:, 1]

# Calculate entropy of probability distribution
ent = entropy(prob)

# Calculate energy of system
energy = np.sum(features ** 2)

# Make decision to buy or sell based on energy and entropy
if energy > 0.5 and ent > 0.5:
    print("Buy", ticker)
elif energy < 0.5 and ent < 0.5:
    print("Sell", ticker)
else:
    print("Hold", ticker)
